In [9]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("etl").getOrCreate()
#spark.sparkContext.getConf().getAll()
!netstat -anp |grep 4040

tcp6       0      0 :::4040                 :::*                    LISTEN      27266/java          


# Outsider of spark transformation 
# Convert the input CSV to Parquet using a separate script prior loading in spark 


In [15]:
import pandas as pd
#!pip install pyarrow

def write_parquet_file():
    df = pd.read_csv('data.csv')
    df.to_parquet('data.parquet',compression='snappy')

write_parquet_file()

In [63]:
# Read data to temporary view 
parquetFile = spark.read.parquet("data.parquet")
parquetFile.createOrReplaceTempView("source_df")

sql  = "select  * from source_df"
full_data = spark.sql(sql)
full_data.show(3)
full_data.describe()

+--------------------+--------------------+----------+-------+---------+-----------------+
|   anonymous_user_id|                 url|      time|browser|       os|screen_resolution|
+--------------------+--------------------+----------+-------+---------+-----------------+
|b527ad6c-1e79-4ae...|https://preview.h...|1523551855| Chrome|    Linux|          800x600|
|2847753f-df48-436...|https://www.nasa....|1523549796| Chrome|Chrome OS|         1366x768|
|7540e6a6-bd75-4f2...|https://www.nasa....|1523549868| Chrome| Mac OS X|        2560x1440|
+--------------------+--------------------+----------+-------+---------+-----------------+
only showing top 3 rows



DataFrame[summary: string, anonymous_user_id: string, url: string, time: string, browser: string, os: string, screen_resolution: string]

In [61]:
sql1 = """
select 
split(split(url,'utm_medium=')[1] , '&' )[0] as utm_medium ,
split(split(url,'utm_source=')[1] , '&' )[0] as utm_source , 
url  
from source_df

"""

#sql = """
#select split(url,'utm_medium=')[1]  , url  as utm_medium , 
#split(split(url,'utm_campaign=')[1] , '&' )[0] as utm_campaign
#from source_df



url_data = spark.sql(sql1)
#url_data.show(5, False)
url_data.createOrReplaceTempView("url_data_view")

sql_top_medium_source = """
select  
utm_medium , utm_source,
count(url) as count 
from url_data_view
group by utm_medium , utm_source
order by count desc 
"""
top_medium_source = spark.sql(sql_top_medium_source)
top_medium_source.show()

+------------+--------------+-----+
|  utm_medium|    utm_source|count|
+------------+--------------+-----+
|         cpc|        google|52386|
|        null|          null|31931|
|        null|      homepage| 3897|
|       email|      hs_email| 1001|
|  paidsocial|      linkedin|  363|
|         cpc|          null|  161|
|  paidsocial|      facebook|   93|
|       email| hs_automation|   62|
|      social|       twitter|   18|
|paid_listing|      capterra|   17|
|      social|   twitter.com|    6|
|Display%20Ad|  Office%20Kit|    4|
|         cpc|           ppc|    2|
|        null|homepage%C2%A0|    2|
|    Facebook|        Social|    1|
|     adwords|          null|    1|
|         cpc|        carbon|    1|
|         cpc|          bing|    1|
|    referral|        wework|    1|
|       email|      sendgrid|    1|
+------------+--------------+-----+



In [96]:
#sql = "select cast(from_unixtime(time) , 'YYYY-MM-DD HH:MM:SS' )  as date from source_df"
epoch2date = """select 
anonymous_user_id,
to_timestamp(from_unixtime(time), 'yyyy-MM-dd HH:mm:ss') as timestamp ,
to_date(from_unixtime(time), 'yyyy-MM-dd HH:mm:ss') as date 
from source_df"""

transformed_df = spark.sql(epoch2date)
transformed_df.show(3)
transformed_df.createOrReplaceTempView("transformed_df")


+--------------------+-------------------+----------+
|   anonymous_user_id|          timestamp|      date|
+--------------------+-------------------+----------+
|b527ad6c-1e79-4ae...|2018-04-13 02:50:55|2018-04-13|
|2847753f-df48-436...|2018-04-13 02:16:36|2018-04-13|
|7540e6a6-bd75-4f2...|2018-04-13 02:17:48|2018-04-13|
+--------------------+-------------------+----------+
only showing top 3 rows



In [99]:

distinct_users_per_day = """
select 
date ,
count(distinct(anonymous_user_id)) as distinct_users 
from transformed_df
group by date
order by date asc 

"""
s = spark.sql(distinct_users_per_day)
s.show()

+----------+--------------+
|      date|distinct_users|
+----------+--------------+
|2018-04-01|           245|
|2018-04-02|          2414|
|2018-04-03|          3401|
|2018-04-04|          3023|
|2018-04-05|          5398|
|2018-04-06|          4866|
|2018-04-07|          4815|
|2018-04-08|          3941|
|2018-04-09|          4146|
|2018-04-10|          6304|
|2018-04-11|          5796|
|2018-04-12|          5771|
|2018-04-13|          5565|
|2018-04-14|          4179|
|2018-04-15|          1406|
+----------+--------------+

